In [0]:
import datetime

job_name = "job_paytronix"
job_start_timestamp = datetime.datetime.now()
development_Mode = False

tasks = [
    {
        "task_name": "task_bronze_paytronix_mid352_combinedcards",
        "task_metadata": "onboard/bronze_paytronix_mid352_combinedcards.yml", 
        "task_notebook": "/Workspace/Shared/databricks_mdd/mdd/shared/onbard_runner",
        "dependencies":[]
    },
    {
        "task_name": "task_gold_fact_combinedcards",
        "task_metadata": "transform/gold_fact_combinedcards.yml", 
        "task_notebook": "/Workspace/Shared/databricks_mdd/etl/metadata/transform/gold_fact_combinedcards_transform",
        "dependencies":["task_bronze_paytronix_mid352_combinedcards"]
    }
]

In [0]:
%load_ext autoreload
%autoreload 2

from mdd.utils import MDDUtil

DAG = {
    "activities": [],
    "timeoutInSeconds": 43200, # max timeout for the entire pipeline, default to 12 hours
    "concurrency": 0 # max number of notebooks to run concurrently, default to unlimited
}

if type(tasks) == list:
    tasks_list = tasks
else:
    #print(tasks)
    tasks_list = json.loads(tasks)

for task in tasks_list:
    activity = {}
    activity["name"] = task["task_name"]
    activity["args"] = {"task_name": task["task_name"].strip(), 
                        "task_metadata": task['task_metadata'].strip(),
                        "job_name": job_name,
                        "job_start_timestamp": job_start_timestamp,
                        "parent_task_run_id": None,
                        "development_mode": development_Mode,
                        }
    activity["path"] = task["task_notebook"].strip()
    activity["retry"] = 1
    activity["retryIntervalInSeconds"] = 10
    activity["dependencies"] = task["dependencies"]

    DAG["activities"].append(activity)

print(DAG)

job_run_id = MDDUtil.get_run_id(job_name, job_start_timestamp)

# log job started
job_end_timestamp = None
job_status = "started"
error_message = None
MDDUtil.etl_job_update(job_name, job_start_timestamp, job_end_timestamp, job_status, error_message)

try:
    if mssparkutils.notebook.validateDAG(DAG):
        mssparkutils.notebook.runMultiple(DAG, {"displayDAGViaGraphviz": True, "rankdir": "LR"})

    # log job succeeded
    job_status = "succeeded"
except Exception as e:
    #log job failed
    job_status = "failed"
    error_message = str(e)
    raise e
finally:
    job_end_timestamp = datetime.datetime.now()
    MDDUtil.etl_job_update(job_name, job_start_timestamp, job_end_timestamp, job_status, error_message)
